In [2]:
import os
from typing import TypedDict, Annotated, List, Dict
import gradio as gr
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from IPython.display import Image, display
import fitz

c:\Users\PratikTalaviya\Crest-Projects\Document-Processing-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
memory = InMemorySaver()
class State(TypedDict):
    """
    The state for our LangGraph agent.
    """
    # documents: List[str] # List of document identifiers
    chat_history: List[Dict[str, str]]
    query: str
    response: str
    document: Dict[str, str]
    extracted_text: Dict[str, str]
    # is_inter_document_query: bool

In [4]:
load_dotenv(override=True)

GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [5]:
from PIL import ImageEnhance, ImageFilter

def preprocess_image(image):
    # Convert to grayscale
    image = image.convert('L')
    # Increase contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)
    # Apply sharpening
    image = image.filter(ImageFilter.SHARPEN)
    return image

In [48]:
# testing text extraction using paddleocr lib

from paddleocr import PaddleOCR
from PIL import Image

# Initialize PaddleOCR (English only)
ocr = PaddleOCR(lang='en', use_textline_orientation=True)

# Path to image
image_path = "test.png"

# Run OCR
result = ocr.predict(image_path)

# Extract text only
texts = [line[1][0] for page in result for line in page]
print(texts)


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\PratikTalaviya\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 1620.05it/s]
Creating model: ('UVDoc', None)
The model(UVDoc) is not supported to run in MKLDNN mode! Using `paddle` instead!
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\PratikTalaviya\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 697.25it/s]
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in C:\Users\PratikTalaviya\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 552.83it/s]
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically

['n', 'a', 'o', 't', 'o', 'e', 'e', 'e', 'e', 'e', 'e', 'i', 'e', 'e']


In [67]:
# testing Text Extraction of EasyOCR Lib.

import numpy as np
from PIL import Image
import easyocr
import cv2

reader = easyocr.Reader(['en'], gpu=False)
# img = Image.open("JaneSmith.png")  # Use a sample image with clear text
img = cv2.imread("JaneSmith.png")  # Use a sample image with clear text
results = reader.readtext(img)
print(results)
for idx, (bbox, text, confidence) in enumerate(results):
    # bbox has 4 points: top-left, top-right, bottom-right, bottom-left
    # Convert to int
    points = np.array(bbox).astype(int)
    
    # Get x, y, w, h from bounding box
    x_min = np.min(points[:, 0])
    y_min = np.min(points[:, 1])
    x_max = np.max(points[:, 0])
    y_max = np.max(points[:, 1])
    
    # Crop image
    cropped = img[y_min:y_max, x_min:x_max]
    
    # Save each cropped region
    cv2.imwrite(f"crop_{idx}_{text}.png", cropped)
    
    print(f"Cropped '{text}' with confidence {confidence:.2f}")

print("✅ Cropping done!")

[2025-08-14 16:22:27,161] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.
c:\Users\PratikTalaviya\Crest-Projects\Document-Processing-Agent\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[([[np.int32(23), np.int32(27)], [np.int32(225), np.int32(27)], [np.int32(225), np.int32(71)], [np.int32(23), np.int32(71)]], 'California', np.float64(0.9501025053608844)), ([[np.int32(223), np.int32(31)], [np.int32(265), np.int32(31)], [np.int32(265), np.int32(51)], [np.int32(223), np.int32(51)]], 'USA', np.float64(0.9929948625253051)), ([[np.int32(292), np.int32(30)], [np.int32(512), np.int32(30)], [np.int32(512), np.int32(58)], [np.int32(292), np.int32(58)]], 'DRIVER LICENSE', np.float64(0.9995673959370764)), ([[np.int32(532), np.int32(28)], [np.int32(620), np.int32(28)], [np.int32(620), np.int32(52)], [np.int32(532), np.int32(52)]], 'FEDERAL', np.float64(0.9990971087934255)), ([[np.int32(531), np.int32(47)], [np.int32(597), np.int32(47)], [np.int32(597), np.int32(67)], [np.int32(531), np.int32(67)]], 'LIMITS', np.float64(0.9250286788180533)), ([[np.int32(531), np.int32(65)], [np.int32(591), np.int32(65)], [np.int32(591), np.int32(85)], [np.int32(531), np.int32(85)]], 'APPLY', np.fl

In [ ]:
from PIL import Image
import easyocr
import io
import fitz  # PyMuPDF
import numpy as np

reader = easyocr.Reader(['en'], gpu=False)

def extract_text_with_ocr_easyocr(pdf_bytes):
    """
    Extract text from all pages of a PDF (provided as bytes) using EasyOCR.
    """
    pdf_text = ""
    
    # Open PDF from bytes
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        for page_num, page in enumerate(doc):
            # Render page to pixmap at 300 DPI
            pix = page.get_pixmap(dpi=300)
            
            # Convert pixmap to NumPy array
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
            
            # OCR using EasyOCR (pass NumPy array, not PIL)
            results = reader.readtext(img_array, detail=0)
            pdf_text += " ".join(results).strip() + "\n"
            
            # Optional: Display image for verification
            img = Image.fromarray(img_array)
            img.show(title=f"Page {page_num}")
    
    return pdf_text



[2025-08-14 15:08:36,653] [ WARNING] easyocr.py:71 - Using CPU. Note: This module is much faster with a GPU.


In [58]:
# import pytesseract
# from pdf2image import convert_from_bytes

# def extract_text_with_ocr(file_bytes):
#     """Extract text from scanned/image-based PDF using Tesseract OCR."""
#     text = ""
#     images = convert_from_bytes(file_bytes)
#     for img in images:
#         text += pytesseract.image_to_string(img)
#     return text

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import numpy as np

pytesseract.pytesseract.tesseract_cmd = r"C:\Users\PratikTalaviya\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

def extract_text_with_ocr_tesseract(file_bytes):
    """
    Render PDF pages as images using PyMuPDF and extract text via Tesseract OCR.
    """
    text = ""
    # Open PDF
    with fitz.open(stream=file_bytes, filetype="pdf") as doc:
        for page_num, page in enumerate(doc):
            # Render page to a pixmap (RGB)
            pix = page.get_pixmap(alpha=False)
            # img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            # # OCR on image
            # Convert pixmap to NumPy array
            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
            
            # Convert NumPy array to PIL Image
            img = Image.fromarray(img_array)
            img = preprocess_image(img)
            page_text = pytesseract.image_to_string(img)
            text += page_text + "\n"
    return text

In [59]:
def llm_generation(state: State):
    """
    Simulates a call to the Bedrock LLM to generate a final response.
    """
    print("---NODE : LLM GENERATION---")
    print(state["extracted_text"])
    response_prompt = state["extracted_text"]
    prompt = f"Based on the following context: '{response_prompt}', answer the user's query: '{state['query']}'"
    # prompt = state["query"]
    final_response = llm.invoke(prompt)
    
    # Update chat history and return the final response
    chat_history = state.get("chat_history", [])
    chat_history.append({"user": state["query"], "assistant": final_response})
    
    return {"response": final_response, "chat_history": chat_history}

def context_generation(state: State):
    """
    Node to extract text from PDF bytes.
    """
    print("---NODE: PDF TEXT EXTRACTION---")
    
    file_bytes_dict = state.get("document")
    if not file_bytes_dict:
        raise ValueError("No file bytes found in state")
    
    extracted_data = {}

    for file_name, file_bytes in file_bytes_dict.items():
        pdf_text = ""
        with fitz.open(stream=file_bytes, filetype="pdf") as doc:
            for page in doc:
                pdf_text += page.get_text()

        if not pdf_text.strip():
            print(f"[OCR] Running OCR for {file_name}...")
            pdf_text = extract_text_with_ocr_tesseract(file_bytes) 
            # pdf_text = extract_text_with_ocr_easyocr(file_bytes) 

        extracted_data[file_name] = pdf_text

    print(extracted_data)
    # Add extracted text to state
    return {
        # **state,  # Keep previous state keys
        "extracted_text": extracted_data,
        "document": None
    }

def should_extract(state):
    """Condition function: returns next node name based on state."""
    print(state)
    if state.get("document"):  # already has text
        return "extraction"

    return "generation"

In [60]:
memory = InMemorySaver()
workflow = StateGraph(State)

workflow.add_node("generation", llm_generation)
workflow.add_node("extraction", context_generation)

workflow.add_conditional_edges(
    START,
    should_extract,
    {
        "extraction": "extraction",  # go to extraction if no text
        "generation": "generation"   # skip extraction if already done
    }
)
# workflow.add_edge("extraction", "generation")
workflow.add_edge("generation", END)

graph = workflow.compile(checkpointer=memory)

In [69]:
display(Image(graph.get_graph().draw_mermaid_png(max_retries=3, retry_delay=5)))

TypeError: 'module' object is not callable

In [ ]:
# --- Gradio UI Functions ---
session_id = "user-session-1"
config = {"configurable": {"thread_id": "1"}}

initial_state = {
    "document": None,
    "chat_history": [],
    "query": "",
    "response": "",
    "extracted_text": {}
}

graph.update_state(config, values=initial_state, task_id=session_id)

def process_document(files):
    """Function to handle document upload and ingestion."""
    if files is None:
        return "Please upload a document first."
    file_bytes_list = {}
    # Read file bytes
    print(files)
    for file in files:
        file_bytes = ""
        with open(file.name, "rb") as f:
            file_bytes = f.read()
        file_bytes_list[os.path.basename(file.name)] = file_bytes

    current_state = graph.get_state(config)
    state_dict = dict(current_state.values)

    # Now you can modify it
    state_dict["document"] = file_bytes_list
    print(file_bytes_list.keys())
    # Invoke the graph with the updated dict
    text = graph.invoke(state_dict, config)
    # Persist updated state
    # graph.update_state(config, values=state_dict, task_id=session_id)
    return f"Successfully Extracted and Processed Documents {file_bytes_list.keys()}"
    # Assuming the graph node returns extracted text or some processed response
    # return True

def chat_interface(query, history):
    """Function to handle user chat messages."""

    current_state = graph.get_state(config)
    
    state_dict = dict(current_state.values)

    # Now you can modify it
    state_dict["query"] = query
    
    graph_state = graph.invoke(state_dict, config)

    # graph.update_state(config, values=graph_state, task_id=session_id)
    
    # Extract the AIMessage object from the "response" key
    ai_message = graph_state.get("response")

    if hasattr(ai_message, "content"):
        return ai_message.content
    elif isinstance(ai_message, dict) and "content" in ai_message:
        return ai_message["content"]
    else:
        return str(ai_message)

# Create the Gradio interface using gr.Blocks for flexibility
with gr.Blocks(title="LangGraph Document Agent") as demo:
    gr.Markdown("# LangGraph Document Agent")
    gr.Markdown("Upload documents to ingest them, then chat with the agent.")
    
    with gr.Row():
        # file_upload = gr.File(label="Upload Document")
        file_upload = gr.File(
        label="Upload Documents",
        file_count="multiple"
        # file_types=[".pdf", ".docx", ".txt"],  # Optional: restrict file types
        )
        process_button = gr.Button("Process Document")
    
    ingestion_output = gr.Textbox(label="Document Processing Status")

    process_button.click(
        fn=process_document,
        inputs=file_upload,
        outputs=ingestion_output
    )

    chatbot = gr.ChatInterface(
        fn=chat_interface,
        examples=["What is the content of document1.txt?", "Compare the names across all documents.", "How is document2.pdf different from document3.docx?"]
    )

# Launch the Gradio app
demo.launch()

{'document': None, 'chat_history': [], 'query': '', 'response': '', 'extracted_text': {}}


c:\Users\PratikTalaviya\Crest-Projects\Document-Processing-Agent\.venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


['C:\\Users\\PratikTalaviya\\AppData\\Local\\Temp\\gradio\\d88061b0f440b28321af8776afd91726d4273959a1c480ef101eb27245de0076\\123690Store Pack.pdf', 'C:\\Users\\PratikTalaviya\\AppData\\Local\\Temp\\gradio\\db56fa54574e40991a0dde4adb6e6fd6b233a3f79b46d9bbb6c129c9a31bec4a\\123690BILL OF SALE.pdf', 'C:\\Users\\PratikTalaviya\\AppData\\Local\\Temp\\gradio\\cbd75bb95b07dbcdc01103b7cd70afc596eefce7c3d1d4ee5f90766c80c3392c\\123690GA MV-12020.pdf']
dict_keys(['123690Store Pack.pdf', '123690BILL OF SALE.pdf', '123690GA MV-12020.pdf'])
{'chat_history': [], 'query': '', 'response': '', 'document': {'123690Store Pack.pdf': b'%PDF-1.7\n\n4 0 obj\n<<\n/BitsPerComponent 8\n/ColorSpace /DeviceRGB\n/Filter /DCTDecode\n/Height 3023\n/Length 112692\n/Subtype /Image\n/Type /XObject\n/Width 2319\n>>\nstream\n\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\r\t\n\x0b\n\x08\r\x0b\x0b\x0b\x0f\x0e\r\x10\x14!\x15\x14\x12\x12\x14(\x1d\x1e\x18!0*21/*.-4;K@48G9-.BYBGNPTUT3?]c\\RbKSTQ

In [ ]:
tesseract_Output = {'License.pdf': 'GEORGI\n\nSEES LICENSE\ner\n\nAb DL\nDRIVER\'S LICENSE\n\n44 DL NO. 999999999 3 vos\n\n» cass ab Exp. —.\n2 IMA GEORGIA\n1 SAMPLE\n\n123 ‘STREET.\n* ANY TOVI, GA Se000\n\n se-nest NONE\n\nga enn NONE\n\naaiss 02/22/2018\n\n16 sex F 18 eves BRO\ntenet 5\'-05" 17wor 165Ib\n\nUSA\nGA\n\n\n'}

easyOCR_Output = {'License.pdf': 'USA GEORGIA DL G4 Driver\'$ LICEnse DRIVER\'S LICENSE Governor: 201 4d DL NO. 999999999 3 DOB 02/22/1954 CLASs C Ab EXP 02/22/2028 2 IMA GEORGIA 4 SAMPLE 4 8 123 MAIN STREET 3 ANYTOWN; GA 39999 ANYCOUNTY 12 REST NONE 1 9a END NONE 4a ISS 02/22/2018 15 SEX F 18 EYES BRO 16 HGT 5\'-05" 17 WGT 165 Ib 02R2185 SIA\n'}

In [ ]:
{'123690Store Pack.pdf': "PEACHTREE AUTO INSURANCE MOTOR VEHICLE INSURANCE POLICY Policy Number: 1P-2Q3RASST Policy Period: March 2024 to March 15,2025 Issue Date: March 15, 2024 POLICYHOLDER INFORMATION Name: SUSAN C. WILLIAMS Date of Birth: 25/09/1995 Driver's License: GA789123456 Phone: (229) 555-7890 Address: 202 Hilltop Rd, Valdosta, GA 31601 VEHICLE INFORMATION VIN: 5XXFW4E68RP123456 Year: 2023 Make: Kia Model: SORENTO Body Style: Sedan Color: Black COVERAGE DETAILS Coverage Type Limit Deductible Premium Bodily Injury Liability S100,000/5300,000 NIA S456.00 Property Damage Liability S50,000 NIA 8234.00 Comprehensive Actual Cash Value S500 S289.00 Collision Actual Cash Value S500 S467.00 Total Annual Premium: $1,446.00 Peachtree Auto Insurance Company 1500 Peachtree Street; Atlanta, GA 30309 (800) 555-PEACE 15,\n", '123690Compliance Pack.pdf': 'CaliforniaUSA DRIVER LICENSE FEDERAL LIMITS APPLY DL GA789123456 CLASS EXP 03/11/2029 END NONE LN SUSAN C FN WILLIAMS 123 SAMPLE STREET 3 SACRAMENTO CA 01234 DOB 25/09/1995 rsTR NONE pokor Jane Dbe SEX HAIR BRO ETES BRO hGt 506 WGt 170 Ib I55 dD 1234567890127456 DDIMMIYYYY\n', '123690BILL OF SALE.pdf': 'Have a question? Visit our website at http://dor.georgia.gov/motor-vehicles or scan the QR code above for more information. \nT-7 (Revised 9-2022)\nWeb and MV Manual\nGeorgia Department of Revenue - Motor Vehicle Division \nBill of Sale \nANY CORRECTION OR ALTERATION WILL VOID THIS FORM \nPurpose of this form: This form is to be used to provide evidence that a transaction between the purchaser(s)/transferee(s) and seller(s)/transferor(s) has taken \nplace and that the odometer reading has been declared by the vehicle’s seller(s) and acknowledged by the vehicle’s buyer(s). \nCompleting this form: This form must be completed in its entirety, legibly printed or typed. \nSection A: Record the vehicle’s information. Please note: Federal regulations require the seller(s)/transferor(s) to disclose the odometer reading and the \npurchaser(s)/transferee(s) to acknowledge the odometer reading upon the transfer of ownership of a vehicle that is not exempt from odometer disclosure \nrequirements. Twenty (20) model years old and older vehicles are exempt from odometer disclosure requirements. If a vehicle is twenty (20) model years old or \nolder, the seller(s)/transferor(s) may enter the word “exempt” in the space provided for the odometer reading. \nSection B: Complete the seller(s)’/transferor(s)’ information. If a court has ordered the sale of the vehicle, record the court’s case number. If the seller/transferor \nis a business, provide Georgia Tax Identification Number and Georgia Sales Tax Number. \nSection C: Provide the purchaser(s)’/transferee(s)’ information. \nSection D: Record, if any, lienholders, security interest holders or encumbrance holders. \nSection E: Certify all statements are true and accurate. \nHow to submit this form: Submit this form along with required documents and applicable tax to the county tag office in the county where the primary purchaser \nresides. Please refer to https://mvd.dor.ga.gov/motor/tagoffices/selecttagoffice.aspx to locate the county tag office in the primary purchaser’s county of \nresidence. \nRequired documents: A court order is required if a court has ordered the sale of the vehicle. A Georgia certificate of title is required if the vehicle being \nsold/transferred is a 1986 or newer model year motor vehicle, motorcycle, camper, travel trailer, or car/tow dolly weighing 2,001 pounds or more; or a 1963 or \nnewer model year mobile home. \nApplicable tax: If the vehicle requires a title, go to https://dor.georgia.gov/drives-e-services and click on the Title Ad Valorem Tax Estimator link to get an \nestimated Title Ad Valorem Tax (TAVT) amount based on the value of the vehicle. Note: Trailers are not subject to TAVT. \nA \nVEHICLE INFORMATION \nVehicle Identification No. (VIN): \nYear: \n \nMake: \nModel: \nOdometer Reading: \n \n(no tenths) \nDate of Sale/Transfer: \n/ \n/ \nPurchase Price: \nB \nSELLER(S) / TRANSFEROR(S) INFORMATION \nCourt Order Case No.: \nIf Court Ordered Sale\nGeorgia Tax ID No.: \nIf Business \nGeorgia Sales Tax No.: \nIf Business \nSeller(s)’/Transferor(s)’ \nFull Legal Name(s): \nPrimary \nSecondary \nMailing Address: \nStreet No. \nStreet Name \nApt./Suite No.  \nCity: \nState: \nZIP Code: \nCounty: \nC \nPURCHASER(S) / TRANSFEREE(S) INFORMATION \nPurchaser(s)’/Transferee(s)’ \nFull Legal Name(s): \nPrimary \nSecondary \nMailing Address: \nStreet No. \nStreet Name \nApt./Suite No.  \nCity: \nState: \nZIP Code: \nCounty: \nD \nLIENHOLDER OR SECURITY INTEREST HOLDER INFORMATION \nLienholder’s or Security \nInterest Holder’s Name: \nMailing Address: \nStreet No. \nStreet Name \nApt./Suite No.  \nCity: \nState: \nZIP Code: \nTelephone No.: \nE \nCERTIFICATION \nI/we, the seller(s)/transferor(s) named in Section B, sold or transferred the vehicle described herein to the buyer(s)/transferee(s) named in Section C. I/we \nhereby certify that to the best of my/our knowledge the odometer reading reflects the total actual mileage of the vehicle unless otherwise indicated. I/we further \nstate that there are no liens, security interests, or encumbrances on this vehicle except as listed in Section D. \nSeller(s)’/Transferor(s)’ \nSignature(s): \nPrimary \nDate \nSecondary \nDate \nBuyer(s)’/Transferee(s)’ \nSignature(s): \nPrimary \nDate \nSecondary \nDate \n5\nX\nX\nF\nW\n4\nE\n6\n8\nR\nP\n1\n2\n3\n4\n5\n6\n2023\nKia\nSorento\n8,000\n32,000\nSouth Georgia Auto\n500\nBroad Ave\nAlbany\nGA\n31701\nSusan C. Williams\n202\nHilltop Rd\nValdosta\nGA\n31601\nChase Bank                                                        INSURANCE POLICY NUMBER: 1P2Q3R4S5T  \n400\nMain St\nValdosta\nGA\n31601\n', '123690GA MV-12020.pdf': 'MV-1 (Revised 6-2020)\nGeorgia Department of Revenue - Motor Vehicle Division \nForm MV-1 Motor Vehicle Title Application \nFor instructions on how to complete this form see page 2.\nA \nVEHICLE INFORMATION \nVehicle ID (VIN): _____________________________  Current Title #:  _________________________ _  Year: \n  ______ \n _ \nMake:       \n    _____________________________  Current Title’s State of Issue:  _______________ _  Color:   \n   ______ \n _ \nModel:        \n    _____________________________  GA County of Residence:      ________________ _  Cylinders:  ______  \n _ \nBody Style:          _____________________________  District #:      __________________________  __  Fuel Type: ______ \n _ \nOdometer Exceptions: \n EXEMPT  \n Exceeds Mechanical Limits of Odometer  \n Not the Actual Mileage, Warning Odometer Discrepancy \nOdometer Reading:  _____________________________  Date Purchased:  _____________________________   \nCOMPLETE FOR ALL COMMERCIAL VEHICLES \nGross Vehicle Weight & Load:  __________________________         Straight Truck? \n Yes \n No           Used for Hire?  \n Yes \n No \nType of Trailer Pulled?  _____________________  Product Hauled?  ____________________  Is this a Farm Vehicle?  \n Yes \n No \nB \nOWNER INFORMATION \nNumber of Owners: _______                Leased Vehicle:  \n No \n Yes (If yes, complete Section D)  \nIf purchased from an out-of-state business, did you pick up the vehicle out of state?  \n Yes \n No \n*Owner’s signature below warrants: I do solemnly swear or affirm under criminal penalty of a felony for fraudulent use of a false or fictitious\nname or address or for making a material false statement punishable by fine up to $5,000 or by imprisonment of up to five years, or both that\nthe statements contained herein are true and accurate.\nOWNER # 1 \nFull Legal Name:  ____________________________________________  Driver’s License #:  __________________  State:  ____ _ \nDate of Birth:  ___________  E-mail Address:  __________________________________________  Phone #:  _________________ \nBusiness Name:  ____________________________________________  Name of Agent:  ______________________________  _ \nAddress:  ________ ______________________________________________________________________________________ \nMailing Address:  _  _______________________________________________________________________________________ \n*Signature of Owner 1 or Business Agent:  ___________________________________________________  Date:  ________________\nOWNER # 2 \nFull Legal Name:  ____________________________________________  Driver’s License #:  __________________  State:  ____ _ \nDate of Birth:  ___________  E-mail Address:  __________________________________________  Phone #:  _________________ \nBusiness Name:  ____________________________________________  Name of Agent:  ________________________________ \nAddress:  ________ ______________________________________________________________________________________ \nMailing Address:  _  _______________________________________________________________________________________ \n*Signature of Owner 2 or Business Agent:  ___________________________________________________  Date:  ________________\nC \nSELLER INFORMATION \nD \nLESSEE INFORMATION \nGA Dealer’s/Bank’s 12-Digit Customer ID # (If applicable): \n__ __ __ __ __ __ __ __ __ __ __ __   \nSeller’s GA Sales Tax #: \n__ __ __ __ __ __ __ __ __ \nFull Legal Name or Business Name and Address: \n_________________________________________________ \n_________________________________________________ \n_________________________________________________ \nIf Georgia Seller, County Name:  _________________________ \nDirectly Financed Dealer Sale: \n Yes \n No \nDriver’s License # (If individual):  __________________________ \nLessee’s Full Legal Name & Address or Business Lessee’s Full Name \n& Address: \n___________________________________________________ \n___________________________________________________ \n___________________________________________________ \nLessee’s GA County Name:  ______________________________ \nLessee’s Phone Number:      ______________________________ \nE \n-\nSECURITY INTEREST OR LIENHOLDER INFORMATION - Attach any information on additional lienholders. \n12-Digit ELT ID #:  __ __ __ __ __ __ __ __ __ __ __ __   Name:  _____________________________________________  _\nAddress:  _____________________________________________________________________________________________ _\n12-Digit ELT ID #:  __ __ __ __ __ __ __ __ __ __ __ __   Name:  _____________________________________________  _\nAddress:  _____________________________________________________________________________________________ _\nF \n-\n-\n-\n-\n-\nATTORNEY-IN-FACT INFORMATION - Attach original Power of Attorney if title is to be mailed to attorney in fact. \nName:  _    _____________________________________________________________________________________________ \nMailing Address:  _____ ___________________________________________________________________________________ \nPhone Number:  ___________________________  E-mail Address:  _________________________________________________ \n5XXFW4E68RP123456\nGA22334455\n2023\nKia\nBlack\nSorento\nLowndes\n4\nSUV\nGasoline\n8000\n3/8/2025\n1\nSusan C. Williams\nGA789123456\nGA\n1995-09-25\nsusan.williams@example.com\n(229) 555-7890\n202 Hilltop Rd, Valdosta, GA 31601\n202 Hilltop Rd, Valdosta, GA 31601\n4\n5\n6\n7\n8\n9\n1\n2\n3\n4\n5\n6\n1\n2\n3\n5\n6\n7\n9\n8\n4\nSouth Georgia Auto\n500 Broad Ave, Albany, GA 31701\nDougherty\n4\n6\n5\n4\n3\n2\n1\n0\n9\n8\n7\n6\n5\nChase Bank\n400 Main St, Valdosta, GA 31601\n \n \nGeorgia Department of Revenue - Motor Vehicle Division \nForm MV-1 Motor Vehicle Title Application  \nINSTRUCTION PAGE \nPurpose of this form: This form is to be used when applying for a tag and title and must be signed by all owners in Section B. \nHow to submit this form: This form must be completed in its entirety, legibly printed or typed, and submitted along with all required documents \nto the county tag office in the county where you reside or to the Department of Revenue (DOR), when applicable. Please refer to \nhttps://dor.georgia.gov to locate the county tag office in your county of residence. \nA \nVEHICLE INFORMATON \nThis section must be completed in its entirety.  If you do not know the district in which you live, please check with your county tag office.    \nInclude all the requested information: vehicle identification number (VIN), make of vehicle, model of vehicle, body style, current title number, \ncurrent title’s state of issue, Georgia county of residence, district number (if known), year of vehicle, color, cylinders of vehicle, fuel type, and \nodometer information including whether exempt, exceeds mechanical limits, not actual mileage.   \nAlso include odometer reading and date purchased.  \nCOMPLETE FOR ALL COMMERCIAL VEHICLES \nThis section must be completed for all requests concerning a commercial vehicle. \nB \nOWNER INFORMATION \nList the number of owners, whether the vehicle is leased, and if it was purchased out of state. \nAll owners listed on the title must sign this form.  By signing this form you are agreeing to the following: \n*Owner’s signature below warrants: I do solemnly swear or affirm under criminal penalty of a felony for fraudulent use of a false or fictitious\nname or address or for making a material false statement punishable by fine up to $5,000 or by imprisonment of up to five years, or both\nthat the statements contained herein are true and accurate.\nOWNER # 1  \nFor owner number one: \n•\nIf an individual, provide the full legal name, driver’s license number, state of issuance, date of birth, e-mail address, telephone\nnumber, address, and mailing address (if applicable).\n•\nIf a business, provide the e-mail address, telephone number, business name, name of the signer, address, and mailing address (if\napplicable).\n•\nSignature is required.\nOWNER # 2 \nFor owner number two: \n•\nIf an individual, provide the full legal name, driver’s license number, state of issuance, date of birth, e-mail address, telephone\nnumber, address, and mailing address (if applicable).\n•\nIf a business, provide the e-mail address, telephone number, business name, name of the signer, address, and mailing address (if\napplicable).\n•\nSignature is required.\nC \nSELLER INFORMATION \nD \nLESSEE INFORMATION \nProvide: \n•\nGeorgia dealer’s or bank’s 12-digit customer identification\nnumber (if applicable)\n•\nSeller’s Georgia sales tax number\n•\nFull legal name or business name and address\n•\nGeorgia county name (if applicable)\n•\nWhether the vehicle was directly financed by the dealer\nProvide: \n•\nLessee’s driver’s license number (if individual)\n•\nLessee’s full legal name and address or Business Lessee’s\nfull name and address\n•\nLessee’s Georgia county name\n•\nLessee’s phone number\nE \n-\nSECURITY INTEREST OR LIENHOLDER INFORMATION - Attach any information on additional lienholders. \nList the following for the first two security interest or lienholders (attach any additional lienholder information to this form): \n•\n12-digit customer identification number\n•\nName\n•\nAddress\nF \n-\n-\n-\n-\n-\nATTORNEY-IN-FACT INFORMATION - Attach original Power of Attorney if title is to be mailed to attorney in fact. \nIf using a Power of Attorney, attach the Power of Attorney and fill in their: \n•\nName\n•\nMailing address\n•\nPhone number\n•\nE-mail address\n'}
